# Imports and Helper Functions

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from scipy.optimize import linprog
from functools import reduce
from math import log, sqrt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle as pkl
import time
import torch

In [2]:
def rotate(l, n):
    return l[n:] + l[:n]
def tensor_sum(list_of_lists):
    # return a tensor of size len(list_of_lists[0]) x ... x len(list_of_lists[-1]) where the value at index (i_1, ..., i_n) is the sum of the values at the same index in the lists
    shape = tuple(len(lst) for lst in list_of_lists)
    tensor = np.zeros(shape)
    idxes = list(range(len(shape)))
    for idx, l in enumerate(list_of_lists):
        tensor += np.transpose(np.tensordot(l, np.ones(shape[idx+1:] + shape[:idx]), axes=0), axes = rotate(idxes, -idx))
    return tensor
def create_cost_tensor(list_list_of_lists):
    shape = tuple(len(lst) for lst in list_list_of_lists[0])
    res = np.zeros(shape)
    for list_of_lists in list_list_of_lists:
        tensor = tensor_sum(list_of_lists)
        res += (tensor / len(shape)) ** 2
    return res, [1/n for n in shape for _ in range(n)]

def ravel_index(dim, index, shape):
    return sum([shape[i] for i in range(dim)]) + index
def convert_to_list(U, shape):
    U_list = []
    i = 0
    for s in shape:
        U_list.append(U[i:i+s])
        i += s
    return U_list
def marginal_k(X, k):
    # marginalize tensor X over dimension k
    return np.sum(X, axis=tuple(axis for axis in range(X.ndim) if axis != k))
def get_marginal_k(p, k, shape):
    return p[ravel_index(k, 0, shape): ravel_index(k, shape[k], shape)]
def rho(a, b):
    return b - a + a * np.log(a / b)
def projection(X, p):
    V = X.copy()
    m, shape = X.ndim, X.shape
    idxes = list(range(len(shape)))
    for r in range(m):
        X_r = np.minimum(get_marginal_k(p, r, shape) / marginal_k(V, r), 1)
        V *= np.transpose(np.tensordot(X_r, np.ones(shape[r+1:] + shape[:r]), axes=0), axes = rotate(idxes, -r))

    err_list = [get_marginal_k(p, r, shape) - marginal_k(V, r) for r in range(m)]
    V += reduce(np.multiply, np.ix_(*err_list)) / (np.abs(err_list[-1]).sum() ** (m-1))
    return V
def solve_lp(costs, target_mu):
    shape = costs.shape
    A = np.zeros((np.sum(shape), np.prod(shape)))
    for j, s in enumerate(shape):
        for i in range(s):
            for idx in product(*map(range, shape)):
                if idx[j] == i:
                    A[ravel_index(j, i, shape), np.ravel_multi_index(idx, shape)] = 1
    c = costs.flatten()
    res = linprog(c, A_eq=A, b_eq=target_mu, bounds=[0, 1], options = {"tol": 1e-8})
    return res.fun, res.x.reshape(shape)

def binary_search(a, b, f, delta = 1e-3):
    while True:
        c = (a+b)/2
        if f(c+delta) < f(c):
            a = c
        else:
            b = c
        if abs(a-b) <= delta:
            break
    return (a+b)/2

def larger_root_qr(a, b, c):
    delta = (b**2) - (4 * a*c)
    assert delta >= 0
    ans1 = (-b - sqrt(delta))/(2 * a)
    ans2 = (-b + sqrt(delta))/(2 * a)
    return max(ans1, ans2)

def A_stable(costs, eta):
    U_tmp = - costs * eta
    max_U_tmp = U_tmp.max().max()
    U_tmp = np.maximum(U_tmp, max_U_tmp - MAX_DIGIT)
    min_U_tmp = U_tmp.min().min()
    return np.exp(U_tmp - min_U_tmp), min_U_tmp

In [3]:
def solve_greenkhorn(costs, target_mu, epsilon=1e-2, verbose = 0, cost_scale = 1, iter_gap = 100):
    """solve using Greenkhorn's algorithm

    Args:
        costs (array)
        target_mu (list): target probability
        epsilon (float): precision. Defaults to 1e-2.
        verbose (int, optional): print intermediate results if verbose >= 2. Defaults to 0.
        cost_scale (int, optional): divide the cost matrix by cost_scale before calculating, scale back in the end. Defaults to 1.
        iter_gap (int, optional): print results every `iter_gap` iterations. Defaults to 100.

    Returns:
        _type_: _description_
    """
    costs /= cost_scale
    shape = costs.shape
    eta = 4 * sum([log (n) for n in shape]) / epsilon
    epsilon_prime = epsilon / 8 / costs.max()
    min_cost = costs.min()
    A_stable = np.exp(-eta * (costs - min_cost))
    A = np.exp(-eta * min_cost) * A_stable
    m = [np.zeros(s) for s in shape]
    B = A_stable / np.sum(np.abs(A_stable))
    def dist(B):
        return sum(np.sum(np.abs(marginal_k(B, i) - get_marginal_k(target_mu, i, shape))) for i in range(B.ndim))
    iter = 0
    if verbose >= 2: obj_list = []
    if verbose >= 2: print("epsilon_prime: ", epsilon_prime)
    while dist(B) > epsilon_prime:
        iter += 1
        max_v = []
        buffer = []
        for k in range(B.ndim):
            marginal = marginal_k(B, k)
            tmp = rho(marginal, get_marginal_k(target_mu, k, shape))
            im = np.argmax(tmp)
            max_v.append(tmp[im])
            buffer.append((marginal[im], im))
        kmax = np.argmax(max_v)
        rki, imax = buffer[kmax]
        m[kmax][imax] += np.log(target_mu[ravel_index(kmax, imax, shape)] / rki)
        if verbose >= 100: print("tensor_sum max", tensor_sum(m).max(), tensor_sum(m).min())
        B = A * np.exp(tensor_sum(m))
        if verbose >= 2 and iter % iter_gap == 0:
            obj = np.sum(projection(B, target_mu) * costs) * cost_scale
            print("iter: ", iter, "dist: ", dist(B), "obj: ", obj)
            obj_list.append(obj)
    if verbose >= 2:
        plt.plot(np.array(range(len(obj_list))) * iter_gap, obj_list)
        plt.title("Primal objective over iterations")
        plt.show()
    weights = projection(B, target_mu)
    return np.sum(weights * costs) * cost_scale, weights

In [4]:
def solve_sinkhorn(costs, target_mu, epsilon=1e-2, verbose = 0, cost_scale = 1, iter_gap = 100):
    """solve using Greenkhorn's algorithm

    Args:
        costs (array)
        target_mu (list): target probability
        epsilon (float): precision. Defaults to 1e-2.
        verbose (int, optional): print intermediate results if verbose >= 2. Defaults to 0.
        cost_scale (int, optional): divide the cost matrix by cost_scale before calculating, scale back in the end. Defaults to 1.
        iter_gap (int, optional): print results every `iter_gap` iterations. Defaults to 100.

    Returns:
        _type_: _description_
    """
    costs /= cost_scale
    shape = costs.shape
    M = len(shape)
    print("shape: ", shape)
    eta = 4 * sum([log (n) for n in shape]) / epsilon
    epsilon_prime = epsilon / 8 / costs.max()
    min_cost = costs.min()
    A_stable = np.exp(-eta * (costs - min_cost))
    A = np.exp(-eta * min_cost) * A_stable
    m = [np.zeros(s) for s in shape]
    B = A_stable / np.sum(np.abs(A_stable))
    def dist(B):
        return sum(np.sum(np.abs(marginal_k(B, i) - get_marginal_k(target_mu, i, shape))) for i in range(M))
    iter = 0
    if verbose >= 2: obj_list = []
    if verbose >= 2: print("epsilon_prime: ", epsilon_prime)
    def stable_update():
        for k in range(M):
            exponents = tensor_sum(m) - costs
            idxes = list(range(M))
            idxes.remove(k)
            tmp_max = exponents.max(axis = tuple(idxes))
            print("k: ", k, "tmp_max: ", tmp_max)
            tmp_tensor = torch.from_numpy(tmp_max).reshape(*([1]*k + [-1] + [1]*(M - k - 1))).expand(*shape).numpy()
            print("m[k] updates: ", np.log(get_marginal_k(target_mu, k, shape)) - np.log(np.exp(exponents - tmp_tensor).sum(axis = tuple(idxes))) - tmp_max)
            m[k] += np.log(get_marginal_k(target_mu, k, shape)) - np.log(np.exp(exponents - tmp_tensor).sum(axis = tuple(idxes))) - tmp_max
    # while dist(B) > epsilon_prime:
    while iter < 100:
        iter += 1
        stable_update()
        # for k in range(M):
        #     ratio = get_marginal_k(target_mu, k, shape)/marginal_k(B, k)
        #     print("k:", k, "ratio max min: ", ratio.max(), ratio.min())
        #     print("marginal(B):", marginal_k(B, k).max(), marginal_k(B, k).min())
        #     m[k] += np.log(get_marginal_k(target_mu, k, shape)) - np.log(marginal_k(B, k))
        if verbose >= 100: print("tensor_sum max", tensor_sum(m).max(), tensor_sum(m).min())
        if verbose >= 2 and iter % iter_gap == 0:
            # primal objective
            B = A * np.exp(tensor_sum(m))
            obj = np.sum(projection(B, target_mu) * costs) * cost_scale
            obj_list.append(obj)
            
            # # lower bound
            # obj = epsilon * sum([sum(get_marginal_k(target_mu, k, shape) * m[k]) for k in range(M)])
            # obj_list.append(obj)
            print("iter: ", iter, "obj: ", obj, "tensor_sum max min: ", tensor_sum(m).max(), tensor_sum(m).min())
    if verbose >= 2:
        plt.plot(np.array(range(len(obj_list))) * iter_gap, obj_list)
        plt.title("Primal objective over iterations")
        plt.show()
    weights = projection(B, target_mu)
    return np.sum(weights * costs) * cost_scale, weights

In [5]:
def get_res_single(tmp_list, solver = solve_lp, return_res = False, print_res = True):
    costs, target_mu = create_cost_tensor(tmp_list)
    lb = np.sum([np.mean([np.mean(t) for t in tmp])**2 for tmp in tmp_list])
    dis, weight = solver(costs, target_mu)
    if print_res: print("mean sq:", lb, "objective value: ", dis)
    if return_res: return lb, dis, weight
    
def get_bootstrap_res(tmp_list, solver = solve_lp, iterations = 100, skip = False):
    res = []
    while len(res) < iterations:
        # sample with replacement from data where data is a list of tuples (group, value)
        # generate a random list containing integers from 0 to len(data) - 1
        L = sum([len(l) for l in tmp_list[0]])
        data_list = []
        for tmp in tmp_list:
            data = []
            for i, l in enumerate(tmp):
                for c in l:
                    data.append((i, c))
            data_list.append(data)
        sample = np.random.randint(0, L, L)
        
        # split into groups
        groups_list = []
        for data in data_list:
            groups = [[] for _ in range(len(tmp_list[0]))]
            for s in sample:
                i, c = data[s]
                groups[i].append(c)
            groups_list.append(groups)
            
        # get results
        flag = False
        for g in groups_list[0]:
            if len(g) == 0:
                flag = True
                break
        if flag:
            if not skip:
                new_groups = []
                for g in groups_list:
                    new_groups.append([l for l in g if len(l) > 0])
                lb, dis, _ = get_res_single(new_groups, solver = solver, return_res = True, print_res = False)
                res.append((lb, dis))
        else:
            lb, dis, _ = get_res_single(groups_list, solver = solver, return_res = True, print_res = False)
            res.append((lb, dis))
    return res

# Data and Solve LP

In [6]:
solver = "sinkhorn"
epsilon = 1
verbose = 2
cost_scale = 1
iter_gap = 1
data_file = "Epitaxialgrowth"

tmp_list = pkl.load(open('data/' + data_file + ".pkl", "rb"))
tmp_list = [[l.astype(np.longdouble) for l in tmp_l] for tmp_l in tmp_list]
tmp_list = [[l - l.mean() for l in tmp_l] for tmp_l in tmp_list]
tic = time.perf_counter()
if solver == "greenkhorn":
    lb, dis, weight = get_res_single(tmp_list, solver = lambda a, b: solve_greenkhorn(a, b, epsilon=epsilon, verbose = verbose, cost_scale = cost_scale, iter_gap = iter_gap), return_res = True)
elif solver == "sinkhorn":
    lb, dis, weight = get_res_single(tmp_list, solver = lambda a, b: solve_sinkhorn(a, b, epsilon=epsilon, verbose = verbose, cost_scale = cost_scale, iter_gap = iter_gap), return_res = True)
toc = time.perf_counter()
print(f"single run: {toc - tic} seconds")

shape:  (161, 37, 36, 19)
epsilon_prime:  0.058367093444878594
k:  0 tmp_max:  [-8.58718802e-07 -8.58691186e-07 -8.58691186e-07 -8.58691186e-07
 -8.58746419e-07 -8.58746419e-07 -8.58718802e-07 -8.58718802e-07
 -8.58718802e-07 -8.58691186e-07 -8.58746419e-07 -8.58718802e-07
 -8.58718802e-07 -8.58718802e-07 -8.58691186e-07 -8.58691186e-07
 -8.58691186e-07 -8.58718802e-07 -8.58691186e-07 -8.58718802e-07
 -8.58718802e-07 -8.58718802e-07 -8.58718802e-07 -8.58691186e-07
 -8.58746419e-07 -8.58746419e-07 -8.58691186e-07 -8.58746419e-07
 -8.58718802e-07 -8.58746419e-07 -8.58691186e-07 -8.58691186e-07
 -8.58732611e-07 -8.58691186e-07 -8.58746419e-07 -8.58746419e-07
 -8.58718802e-07 -8.58718802e-07 -8.58691186e-07 -8.58718802e-07
 -8.58691186e-07 -8.58691186e-07 -8.58718802e-07 -8.58718802e-07
 -8.58742967e-07 -8.58718802e-07 -8.58711898e-07 -8.58746419e-07
 -8.58704994e-07 -8.58691186e-07 -8.58691186e-07 -8.58718802e-07
 -8.58635953e-07 -8.58691186e-07 -8.58718802e-07 -8.58691186e-07
 -8.5871880

KeyboardInterrupt: 